### Test your DynamoDB table

In [ ]:
import boto3
import json
import time
import random 
import string
from decimal import Decimal

# Enable after PIP INSTALL faker
# from faker import Faker

In [ ]:
class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return str(obj)
    return json.JSONEncoder.default(self, obj)

### Get Table name

In [ ]:
# Read Key "context" from local cdk.json file:
with open('../cdk.json') as f:
    data = json.load(f)
    ddb_table = data['context']['env-virtual-assistant-dev01']['dynamodb_table_name']

### Simple Query test

In [ ]:
print(ddb_table)

In [ ]:
# DynamoDB Get all Items
def get_all_items():
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(ddb_table)
    response = table.scan()
    items = response['Items']
    return items

# Print items
items = get_all_items()
ddb_data = json.dumps(items, indent=4, cls=DecimalEncoder)
print(ddb_data)

### Inserting Items one by one

In [ ]:
# Initialize Faker and DynamoDB resource
fake = Faker()
dynamodb = boto3.resource('dynamodb')

# Connect to your DynamoDB table
table_name = ddb_table
table = dynamodb.Table(table_name)

# Function to create a random department store item
def create_random_item(_faker):
    """_summary_

    Args:
        _faker (_type_): _description_

    Returns:
        _type_: _description_
    """
    department = _faker.word(ext_word_list=['Electronics', 'Clothing', 'Home', 'Beauty', 'Sports', 'Books'])
    product_name = _faker.catch_phrase()
    price = Decimal(str(_faker.random_number(digits=2))) + Decimal(str(_faker.random_int(min=5, max=95) / 100))  # Random price between $0.05 and $99.95
    
    # Date:
    date_now = _faker.date_this_year().isoformat()

    return {
        'pk': f'DEPARTMENT#{department}',           # Partition key
        'sk': f'PRODUCT#{date_now}#{_faker.uuid4()}',            # Sort key
        'department': department,
        'product_name': product_name,
        'price': price,
        'stock': _faker.random_int(min=1, max=100),   # Random stock quantity
        'supplier': _faker.company(),
        'rating': Decimal(str(_faker.pyfloat(left_digits=1, right_digits=1, positive=True, max_value=5))),  # Rating between 0.0 and 5.0
        'added_on': date_now
    }

# Insert multiple random items
def insert_random_data(ddb_table, num_items):
    # Get Table:
    table = ddb_table
    
    for _ in range(num_items):
        item = create_random_item()
        response = table.put_item(Item=item)
        print(f"Inserted item: {item}")

    return response


In [ ]:
# Insert 10 random items
insert_random_data(5)

### Inserting rows in Batch

In [ ]:
def put_items(ddb_table, items):
    """_summary_

    Args:
        ddb_table (_type_): _description_
        items (_type_): _description_

    Returns:
        _type_: _description_
    """

    # Initialize DynamoDB resource
    dynamodb = boto3.resource('dynamodb')

    # Table init
    table = dynamodb.Table(ddb_table)
    
    # List to store Partition Key values
    pk_values = []

    # Get current time in unix timestamp
    current_time = int(time.time())
    
    with table.batch_writer() as batch:
        for item in items:
            try:
                # Append key and value to item
                item['sk'] = str(current_time)

                # Upsert all in batch
                batch.put_item(Item=item)

                # Append inserted Partition Key for monitoring
                pk_values.append(item['pk'])
                
            except Exception as e:
                print(f"Error inserting item: {item}. Error: {e}")

    return pk_values

In [ ]:
# Example usage
items = [
    {'pk': '001', 'name': 'John Doe', 'email': 'john.doe@example.com'},
    {'pk': '002', 'name': 'Jane Smith', 'email': 'jane.smith@example.com'},
    {'pk': '003', 'name': 'Bob Johnson', 'email': 'bob.johnson@example.com'},
]

inserted_pk_values = put_items(ddb_table, items)
print(inserted_pk_values)

### Avoiding generic items, to be specific on which KEYS to accept

In [ ]:
def put_items(ddb_table, product_id, product_name, price_gbp, quantity):
    """_summary_

    Args:
        ddb_table (_type_): _description_
        product_id (_type_): _description_
        product_name (_type_): _description_
        price_gbp (_type_): _description_
        quantity (_type_): _description_

    Returns:
        _type_: _description_
    """

    # Initialize DynamoDB resource
    dynamodb = boto3.resource('dynamodb')

    # Table init
    table = dynamodb.Table(ddb_table)
    
    # Print for terminal
    print(f"Inserting product ID: {product_id}")

    # Get current time in unix timestamp. 
    # - Primary Key in DDB is Partition Key (pk) and Sort Key (sk). This is our Order ID.
    current_time = int(time.time())

    # Generate a fake session id
    session_id = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
    
    # Append key and value to item
    item = {}
    item['pk'] = f'product_id#{product_id}#session_id#{session_id}'
    item['sk'] = str(current_time)
    item['product_id'] = product_id
    item['product_name'] = product_name
    item['price_gbp'] = price_gbp
    item['quantity'] = quantity

    # Cast Float to Decimals, for DDB support.
    ddb_data = json.loads(json.dumps(item), parse_float=Decimal)

    # Logging and metrics
    # logger.info(f"Storing product sale with details (DDB Item to insert): {item}")

    try:
        # Insert item in DynamoDB table
        response = table.put_item(Item=ddb_data)
        print(response)

        return {
            'statusCode': 200,
            'body': f'Sale recorded for product ID {product_id}'
        }
    
    except Exception as e:
        return {
            'statusCode': 500,
            'body': f'Error inserting item: {e}'
        }


In [ ]:
# Test put_items function
response = put_items(ddb_table=ddb_table, product_id="p123",product_name="product test train",price_gbp=100.1,quantity=1)
print(response)